In [1]:
import sqlite3
con = sqlite3.connect("JudicialAnalysis_R.db")

cur = con.cursor()

In [2]:
# 把原本裁判書檔按順序反過來
# 從 2023年12月開始
reverse = 0
if reverse:
    import os 

    source_file = os.path.join(
            'C:\\', '課程', '交大','論文','案件整理',
            'json_files.txt'
            )

    with open(source_file, 'r', encoding='utf-8') as f:
        # 2022~2023年判決書有211萬7,454個json檔
        # 故一次讀一個處理，減少記憶體占用
        json_files_R = f.read().strip().split('\n')

    json_files_R.sort(reverse=True)
    json_files_R = [f for f in json_files_R if os.path.isfile(f)]

    source_file_R = os.path.join(
            'C:\\', '課程', '交大','論文','案件整理',
            'json_files_R.txt'
            )
    
    json_files_R_STR = '\n'.join(json_files_R).strip()
    
    with open(source_file_R, 'w', encoding='utf-8') as f:
        # 2022~2023年判決書有211萬7,454個json檔
        # 故一次讀一個處理，減少記憶體占用
        f.read().strip().split('\n')

In [3]:
# 創建 Table
new = 0
if new:
    cur.execute("CREATE TABLE results(jid, date, law, keyword_type, keywords)")
    con.commit()
    con.close()

In [4]:
from ElementExtractor import Judicial_Parser
import os, time, traceback, json, re

In [5]:
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER

In [6]:
# Downloads to ./data.zip (2GB) and extracts to ./data/
# data_utils.download_data_url("./") # iis-ckip
#data_utils.download_data_gdown("./") # gdrive-ckip 已下載就不用再下載

In [7]:
# To use GPU:
#    1. Install tensorflow-gpu (see Installation)
#    2. Set CUDA_VISIBLE_DEVICES environment variable, e.g. os.environ["CUDA_VISIBLE_DEVICES"] = "0"
#    3. Set disable_cuda=False, e.g. ws = WS("./data", disable_cuda=False)

# 採坑心得
# 新版已經沒有 tensorflow-gpu
# 我使用 python 3.7 的環境 (conda create -n ckipcore python=3.7)
# 先安裝 pip install -U ckiptagger[tf,gdown]
# 再改用 Conda install tensorflow 去安裝匹配的新版
# 搭配 disable_cuda=False 參數，從 windows 工作管理員之效能頁籤看起來，確實有使用 GPU

ws = WS("./data", disable_cuda=False)
pos = POS("./data", disable_cuda=False)
ner = NER("./data", disable_cuda=False)

C:\ProgramData\Anaconda3\envs\ckipcore\lib\site-packages\ckiptagger\model_ws.py:106: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)
C:\ProgramData\Anaconda3\envs\ckipcore\lib\site-packages\ckiptagger\model_pos.py:56: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)
C:\ProgramData\Anaconda3\envs\ckipcore\lib\site-packages\ckiptagger\model_ner.py:57: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.n

In [8]:
# 簡繁轉換器 (暫時用不到)
# import opencc
# converter = opencc.OpenCC('t2s.json')

In [13]:
from IPython.display import clear_output

start_time = time.time()
print(f'[現在時間] {time.strftime("%Y-%m-%d %H:%M:%S")}')

source_file = os.path.join(
        'C:\\', '課程', '交大','論文','案件整理',
        'json_files_R.txt'
        )
    
count_file_done_num = 0
clear_output()

# 多工時使用
skip = 0 # 要跳過從第幾開始

print('-'*50)
print(f'>>開始分析...')
print(f'[當前已分析數量] {count_file_done_num}')

with open(source_file, 'r', encoding='utf-8') as f:
    # 2022~2023年判決書有211萬7,454個json檔
    # 故一次讀一個處理，減少記憶體占用
    raw_data = ''
    
    while 1: 
        # 開始逐案讀取資料分析
        file = f.readline()
        count_file_done_num += 1
        
        if '刑事' in file:
            # 篩掉一些比較非刑案的
            if not ('簡易' in file or '民事' in file or '補償' in file or '憲法' in file or '商業' in file):
                # 開始讀取裁判書內容
                file = file.strip()

                # 顯示進度
                temp_time = time.time()
                elapsed_time = temp_time - start_time
                hours = int(elapsed_time // 3600)  # 計算小時數
                minutes = int((elapsed_time % 3600) // 60)  # 計算分鐘數
                seconds = int(elapsed_time % 60)  # 計算剩餘秒數
                
                clear_output()
                print('-'*50)
                print(f'>>開始分析...')
                print(f'[當前已分析數量] {count_file_done_num}')
                print(f'[累計耗時] {hours}:{minutes}:{seconds}')
                print(f'[當前檔案] {file}')
                
                # 確定 file 存在
                if not os.path.isfile(file):
                    print(f'[並非正常檔案] 跳過...')
                    time.sleep(1)
                    continue
                
                with open(file, 'r', encoding='utf-8') as jf:
                    json_data = json.load(jf)
                
                # 資料清洗
                parser = Judicial_Parser(json_data)
                
                jid = parser.jid.strip().replace(' ', '')
                date = parser.dateSTR.strip().replace(' ', '')
                raw = parser.raw
                text = parser.text
                text_u = parser.text_u
                
                # 確定是否已經在資料庫內
                check_res = cur.execute(f"SELECT jid FROM results WHERE jid='{jid}'")
                temp = check_res.fetchone()
                if temp:
                    print(f'[JID] {jid}')
                    print('>>此JID發現資料庫已有，跳過')
                    continue
                    
                # 沒有在資料庫內才處理
                # Input text
                print('[WS] 分析中')
                sentence_list = text_u.split('。')
                word_sentence_list = ws(
                    sentence_list,
                    # sentence_segmentation = True, # To consider delimiters
                    # segment_delimiter_set = {",", "。", ":", "?", "!", ";"}), # This is the defualt set of delimiters
                    # recommend_dictionary = dictionary1, # words in this dictionary are encouraged
                    # coerce_dictionary = dictionary2, # words in this dictionary are forced
                )
                print('[POS] 分析中')
                pos_sentence_list = pos(word_sentence_list)
                print('[NER] 分析中')
                entity_sentence_list = ner(word_sentence_list, pos_sentence_list)
                
                # 用 POS 區分的類型
                type_dict = dict()
                for idx, words in enumerate(word_sentence_list):
                    for idx2, w in enumerate(words):
                        temp_type = pos_sentence_list[idx][idx2]
                        type_dict.setdefault(temp_type, set())
                        type_dict[temp_type].add(w)
                
                # 經人工檢查，N(名詞)比較符合需要的有
                # 'Na', 'Nb'
                type_N = [t for t in type_dict.keys() if t.startswith('N')]
                type_N.sort()
                for n in type_N:
                    print('-'*50)
                    print(f'TYPE: {n}')
                    print(type_dict[n])

                # 經人工檢查，V(動詞)比較符合需要的有
                # 'VA', 'VC', 'VD, 'VH', 
                type_V = [t for t in type_dict.keys() if t.startswith('V')]
                type_V.sort()
                for v in type_V:
                    print('-'*50)
                    print(f'TYPE: {v}')
                    print(type_dict[v])
                
                # 擷取法條
                laws = set()
                for e in entity_sentence_list:
                    for ee in e:
                        if ee[2] == 'LAW':
                            laws.add(ee[3])
                print('-'*50)
                print('TYPE: LAW')
                print(laws)
                
                if not laws:
                    print('>>此篇未發現 law 犯罪條文，跳過')
                    continue
                
                # 寫入 Table
                print('-'*50)
                print('>>準備寫入 Table')
                # results(jid, date, law, keyword_type, keywords)
                
                for law in laws:
                    # 初步處理一下 law 標準化
                    law = law.strip().replace(' ', '').upper()
                    
                    # 先從名詞，再來動詞
                    type_range = ['Na', 'Nb', 'VA', 'VC', 'VD', 'VH']
                    for tr in type_range:
                        print('-'*50)
                        print(f'>>準備寫入 {tr}')
                        if tr in type_dict:
                            na_set = {n.strip().replace(' ', '') for n in type_dict[tr] if len(n.strip().replace(' ', '')) > 1}
                            # 發現結果都會有很多,,,可能是 n 裡面本來就有 ,
                            # 所以先轉換一次，去除以後再存
                            na_set = ','.join(na_set).split(',')
                            na_set = [n.upper() for n in na_set if n]
                            na_set = list(set(na_set)) # 去除重複
                            na_set.sort()
                            na_strs = ','.join(na_set).strip().replace(' ', '')
                            while ',,' in na_strs:
                                na_strs = na_strs.replace(',,', ',')
                            to_db_content = f"INSERT INTO results VALUES('{jid}', '{date}', '{law}', '{tr}', '{na_strs}')".strip().replace(':')
                            cur.execute(to_db_content)
                    con.commit()
                    
                    print('-'*50)
                    print('>>完成!!!')
con.close()
end_time = time.time()
elapsed_time = end_time - start_time
hours = int(elapsed_time // 3600)  # 計算小時數
minutes = int((elapsed_time % 3600) // 60)  # 計算分鐘數
seconds = int(elapsed_time % 60)  # 計算剩餘秒數
print(f'[累計耗時] {hours}:{minutes}:{seconds}')

--------------------------------------------------
>>開始分析...
[當前已分析數量] 5181
[累計耗時] 0:2:0
[當前檔案] C:\課程\交大\論文\案件整理\202312\臺灣高雄地方法院刑事\KSDM,110,重附民,9,20231215,1.json
[WS] 分析中
[POS] 分析中
[NER] 分析中
--------------------------------------------------
TYPE: Na
{'書記官', '民事', '正本', '事件', '案件', '民字', '照', '地方', '訴訟法', '銀行法', '時日', '法官', '審判長', '原告', '被告', '刑事'}
--------------------------------------------------
TYPE: Nb
{'黃俊葳', '楊金陵', '陳芸珮', '黃三友', '李美萱', '林豐富', '李家如', '魏偉芬', '陳建文', '陳建名', '林育丞', '盧沛蓁', '黃瀅捷', ' 告', '吳學庸', '張家祥'}
--------------------------------------------------
TYPE: Nc
{'中華民國', '院', '法院', '高雄', '民事庭', '臺灣'}
--------------------------------------------------
TYPE: Nd
{'110年度', '12月', '15日', '112年'}
--------------------------------------------------
TYPE: Nep
{'其'}
--------------------------------------------------
TYPE: Nes
{'前', '本'}
--------------------------------------------------
TYPE: Neu
{'第9', '第504', '第1'}
--------------------------------------------------
TYPE: Nf
{'段',

TypeError: replace() takes at least 2 arguments (1 given)